<a href="https://colab.research.google.com/github/wmjllrh/Group-16/blob/main/Group_16_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up (Libraries, Importing Data, Cleaning Data)

###Importing libraries

In [ ]:
# Imporing libraries
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
from datetime import datetime
import random
%matplotlib inline

###Importing datasets

In [ ]:
# 'Gun Violence Archive' data (for each month of 2021)
jan = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/JAN_2021.csv', encoding = 'latin1')
feb = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/FEB_2021.csv', encoding = 'latin1')
mar = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/MAR_2021.csv', encoding = 'latin1')
apr = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/APR_2021.csv', encoding = 'latin1')
may = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/MAY_2021.csv', encoding = 'latin1')
jun = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/JUN_2021.csv', encoding = 'latin1')
jul = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/JUL_2021.csv', encoding = 'latin1')
aug = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/AUG_2021.csv', encoding = 'latin1')
sep = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/SEPT_2021.csv', encoding = 'latin1')
oct = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/OCT_2021.csv', encoding = 'latin1')
nov = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/NOV_2021.csv', encoding = 'latin1')
dec = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/DEC_2021.csv', encoding = 'latin1')

In [ ]:
# US Census population estimates data (2020-2021)
pop = pd.read_csv ('https://raw.githubusercontent.com/wmjllrh/Group-16/main/US_POP_20_21.csv', encoding = 'latin1')

In [ ]:
# US decennial ethnicity census data [2020: DEC Redistricting Data (PL94-171)]
eth = pd.read_csv ('https://raw.githubusercontent.com/wmjllrh/Group-16/main/DECENNIALPL2020.P1-2022-10-27T094912.csv', encoding = 'latin1', skipinitialspace = True)

In [ ]:
# County Presidential Election Returns 2000-2020 (MIT Election Data and Science Lab, 2018)
election_data = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/countypres_2000-2020.csv', encoding = 'latin1')

In [ ]:
# American Community Survey 2020 5-Year Estimates - Income in the past 12 months (IN 2020 INFLATION-ADJUSTED DOLLARS)
income_data = pd.read_csv('https://raw.githubusercontent.com/wmjllrh/Group-16/main/ACS%202020%20Income%20Data.csv', encoding = 'latin1')

In [ ]:
# Scraping state abbreviations
abbreviations = pd.read_html('https://www.ssa.gov/international/coc-docs/states.html')[0]

###Cleaning and joining datasets

####Election data (2020)

In [ ]:
# Removing all years bar 2020
election_data.drop(election_data[election_data['year'] != 2020].index, inplace = True)

# Removing unnecessary columns
election_data = election_data.drop (columns = [election_data.columns[2], election_data.columns[4], election_data.columns[5], 
                                               election_data.columns[6], election_data.columns[10], election_data.columns[11]])

# Converting 'party' row to column headers
ed = election_data.pivot_table('candidatevotes', ['year', 'state', 'county_name', 'totalvotes'], 'party', aggfunc = 'sum')

# Collapsing rows (so that each row represents a state)
ed = ed.groupby('state').sum()

# Re-creating a 'total votes' column
ed['total_votes'] = ed['DEMOCRAT'] + ed['GREEN'] + ed['LIBERTARIAN'] + ed['OTHER'] + ed['REPUBLICAN']

# Creating percentage vote columns (so results are standardised and comparable)
ed['democrat_pct_2020'] = ed.DEMOCRAT / ed.total_votes * 100
ed['green_pct_2020'] = ed.GREEN / ed.total_votes * 100
ed['libertarian_pct_2020'] = ed.LIBERTARIAN / ed.total_votes * 100
ed['other_pct_2020'] = ed.OTHER / ed.total_votes * 100
ed['republican_pct_2020'] = ed.REPUBLICAN / ed.total_votes * 100

# Retaining only the newly created columns
ed = ed.iloc[:, 6:11]

# Re-ordering columns
ed = ed[['republican_pct_2020', 'democrat_pct_2020','green_pct_2020','libertarian_pct_2020','other_pct_2020']]

####Gun Violence Archive data (2021)

In [ ]:
# Joining GVA data
gva_data = pd.concat ([jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec], ignore_index = True)

# Grouping GVA data by state, then summing the number of mortalities by firearms in each.
gva_data = gva_data.groupby ('State') ['# Killed'].sum().reset_index()

# Renaming the '# Killed' column and 'State' column
gva_data.rename (columns = {'# Killed':'deaths_2021', 'State':'state'}, inplace = True)

# Setting 'state' as the index
gva_data = gva_data.set_index('state')

####US Census population estimates data (2021)

In [ ]:
# Renaming US Census population estimates 'state' and the 2021 population estimate column
pop.rename (columns = {pop.columns[0]: "state", pop.columns[3]: "population_2021"}, inplace = True)

# Removing 2020 population estimates columns. For our analyses, we will use 2021 population estimates.
pop = pop.drop (columns = [pop.columns[1], pop.columns[2]])

# Setting 'state' as the index
pop = pop.set_index('state')

####US decennial ethnicity census data [2020: DEC Redistricting Data (PL94-171)]

In [ ]:
# Renaming columns
eth.rename (columns =
            {eth.columns[0]: "state", eth.columns[1]: "total", eth.columns[3]:
             "white_pop"},
            inplace = True)

# Shift state column rows down by ONE
eth.loc[:, 'state'] = eth.state.shift(+1)

# Removing ethnicity data rows with NaN values and removing commas
eth = eth.dropna()
eth = eth.replace(',','', regex = True)

# Retaining only state, white pop and total columns
eth = eth[['state', 'white_pop', 'total']]

# Setting 'state' as the index
eth = eth.set_index('state')

# Converting remaining columns to numeric
eth = eth.apply(pd.to_numeric)

# Creating a % white column
eth['white_pct_2020'] = eth.white_pop / eth.total * 100

# Dropping unnecessary columns
eth = eth.drop (columns = [eth.columns[0], eth.columns[1]])

####American Community Survey 2020 5-Year Estimates - Income in the past 12 months (IN 2020 INFLATION-ADJUSTED DOLLARS)

In [ ]:
# Renaming columns of interest
income_data.rename (columns =
            {income_data.columns[0]: "state", income_data.columns[2]: "household_less_than_10k", income_data.columns[11]:
             "household_200k_or_more", income_data.columns[12]: "median_household_income"},
            inplace = True)

# Shift state column rows down by TWO
income_data.loc[:, 'state'] = income_data.state.shift(+2)

# Retaining only columns of interest
income_data = income_data[['state', 'household_less_than_10k', 'household_200k_or_more', 'median_household_income']]

# Removing data rows with NaN values and removing commas
income_data = income_data.dropna()
income_data = income_data.replace(',','', regex = True)

# Converting percentage strings to numeric
income_data['household_less_than_10k'] = income_data['household_less_than_10k'].str.rstrip("%").astype(float)
income_data['household_200k_or_more'] = income_data['household_200k_or_more'].str.rstrip("%").astype(float)

# Setting 'state' as the index
income_data = income_data.set_index('state')

# Converting remaining columns to numeric
income_data = income_data.apply(pd.to_numeric)

# Making the index uppercase to match our combine dataframe
income_data.index = income_data.index.str.upper()

####State abbreviations

In [ ]:
# Renaming columns
abbreviations.rename (columns =
            {abbreviations.columns[0]: "state", abbreviations.columns[1]: "code"},
            inplace = True)

# Setting 'state' as the index
abbreviations = abbreviations.set_index('state')

####Combined data

In [ ]:
# Left joining the GVA and Census datasets (keeping only the index values of 'gva_data')
df = gva_data.merge (pop, left_index = True, right_index = True, how = 'left')

# Removing commas from the dataframe and converting the 'population_2021' column to numeric
df = df.replace(',','', regex = True)
df["population_2021"] = pd.to_numeric(df["population_2021"])

# Standardising firearms mortalities to allow comparison across states 
## [creating a 'rate of firearms-related mortalties per 100,000 people' column]
df['deaths_per_100k'] = df.deaths_2021 / df.population_2021 * 100000

# Removing unnecessary 'population' and 'mortality' columns
df = df.drop (columns = [df.columns[0], df.columns[1]])

# Merging ethnicity data
df = df.merge (eth, left_index = True, right_index = True, how = 'left')

# Making the index uppercase, so we may join the election data
df.index = df.index.str.upper()

# Merging 2020 election data
df = df.merge (ed, left_index = True, right_index = True, how = 'left')

# Merging 2020 ACS income estimates data
df = df.merge (income_data, left_index = True, right_index = True, how = 'left')

# Merging state abbreviations
df = df.merge (abbreviations, left_index = True, right_index = True, how = 'left')

#Statistical analyses

##Checking regression assumptions

##Linear regression

##Multiple regression

#Visualisations

##Scatterplots

##Box plots

##Choropleth maps

####Messing around with choropleth maps

In [ ]:
# code for creating choropleth map of USA states
# import plotly library
import plotly
 
# import plotly.express module
# this module is used to create entire figures at once
import plotly.express as px
 
# create figure
fig = px.choropleth(locationmode="USA-states", color=[1], scope="usa")
 
fig.show()

In [ ]:
#code for representing states of USA
#pass list of states in locations
#list will have two-letter abbreviations of states
fig = px.choropleth(df, locations='code', locationmode="USA-states", color='deaths_per_100k', scope="usa")
fig.update_layout(geo = dict(showlakes=False))
 
fig.show()